## Library Imports


In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from statistics import mode
import matplotlib.pyplot as plt
import os, pickle, IPython

from sklearn import datasets, preprocessing 
from sklearn.model_selection import train_test_split
from mlxtend.data import mnist_data
from keras.datasets import mnist

from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
from qiskit.quantum_info import Statevector

from skimage import io, transform, color
from skimage.feature import graycomatrix , graycoprops

%matplotlib widget

def get_current_directory():
    try:
        
        directory = os.path.dirname(os.path.abspath(__file__))

    except:
        
        ip = IPython.get_ipython()
        directory = None
        if '__vsc_ipynb_file__' in ip.user_ns:
            directory = os.path.dirname(ip.user_ns['__vsc_ipynb_file__'])
        
    return directory 


def salvar(a):
    
    script_dir = get_current_directory()
    
    dados_dir = os.path.join(script_dir, 'dados')
    
    os.makedirs(dados_dir, exist_ok=True)
    
    file_path = os.path.join(dados_dir, f'{a}.pickle')
    
    with open(file_path, 'wb') as f:
        pickle.dump(eval(a), f)

def abrir(a):
    
    script_dir = get_current_directory()
    
    file_path = os.path.join(script_dir, 'dados', f'{a}.pickle')
    
    with open(file_path, 'rb') as f:
        return pickle.load(f)
    
def salvar_obj(obj, filename):
    script_dir = get_current_directory()
    dados_dir = os.path.join(script_dir, 'dados')
    os.makedirs(dados_dir, exist_ok=True)
    
    file_path = os.path.join(dados_dir, f'{filename}.pickle')
    
    with open(file_path, 'wb') as f:
        pickle.dump(obj, f)

2024-11-12 17:27:55.201783: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-12 17:27:55.203752: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-12 17:27:55.209501: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-12 17:27:55.218344: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-12 17:27:55.220902: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-12 17:27:55.227896: I tensorflow/core/platform/cpu_feature_gu

## Dataset Loading and Initial Exploration

In [ ]:
(train_X, train_y), (test_X, test_y) = mnist.load_data()

## Making features vectors (X)

In [ ]:
def quantize_h(hue):
    return int(hue // 40)  

def quantize_s(saturation):
    return int(saturation // 0.33)  

def quantize_b(brightness):
    return int(brightness // 0.33)  

angles = [0, np.pi/4, np.pi/2, 3*np.pi/4]

def extract_features(X):
    
    features = []
    
    for idx in range(X.shape[0]):

        img = X[idx].reshape(28, 28)
        

        gray_img = img
        

        hsv_img = color.rgb2hsv(np.stack((gray_img,)*3, axis=-1))  
        hue = hsv_img[:, :, 0] * 360
        saturation = hsv_img[:, :, 1]
        brightness = hsv_img[:, :, 2]
        
        h_quantized = np.vectorize(quantize_h)(hue)
        s_quantized = np.vectorize(quantize_s)(saturation)
        b_quantized = np.vectorize(quantize_b)(brightness)
        
        Q_h = 9 
        Q_s = 3
        G = h_quantized * Q_h + s_quantized * Q_s + b_quantized
        
        
        gray_img_uint = (gray_img * 255).astype(np.uint8)
        glcm = graycomatrix(gray_img_uint, distances=[1], angles=angles)
            
        contrast = graycoprops(glcm, 'contrast')
        correlation = graycoprops(glcm, 'correlation')
        energy = graycoprops(glcm, 'energy')
        
        
        entropy = -np.sum(glcm * np.log2(glcm + (glcm == 0)), axis=(0, 1))
        
        glcm_features = []
        for prop in [contrast, correlation, energy, entropy]:
            mean = np.mean(prop)
            variance = np.var(prop)
            glcm_features.extend([mean, variance])
        
        G_histogram, _ = np.histogram(G, bins=np.arange(73))
        
        feature_vector = np.concatenate([G_histogram, glcm_features])
        features.append(feature_vector)
    
    return np.array(features)

X = extract_features(train_X)

In [11]:
X = abrir('features_train')
y = abrir('train_y')
train_X = abrir('train_X')
test_X = abrir('test_X')
test_y = abrir('test_y')
features_test = abrir('features_test')

M = len(X[0])

In [9]:
print('Extracted features shape:', X.shape)

Extracted features shape: (60000, 80)


In [20]:
X_test = extract_features(test_X)

## Data Visualization with Pair Plot


## Data Normalization


In [17]:
min_max_scaler = preprocessing.MinMaxScaler(feature_range=(0, 1))
min_max_scaler.fit(X)
X_normalized = min_max_scaler.transform(X)

## Test Sample Selection and Quantum State Preparation


In [ ]:
test_index = int(np.random.rand()*len(test_X)) 
phi_test = test_X[test_index]

# Create the encoded feature vector
for i in range(M):
    phi_i = [np.sqrt(phi_test[i]), np.sqrt(1- phi_test[i])]
    if i == 0:
        phi = phi_i
    else:
        phi = np.kron(phi_i, phi)    


MemoryError: Unable to allocate 57.4 GiB for an array with shape (2, 32, 28, 17210368) and data type float16